### Name : Serik Batyrkhan 

### Group : BDA-2207

In [4]:
!pip install spacy

   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
    --------------------------------------- 0.3/12.2 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.2 MB 4.2 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/12.2 MB 4.9 MB/s eta 0:00:03
   ---------------- ----------------------- 5.0/12.2 MB 7.4 MB/s eta 0:00:01
   ---------------------- ----------------- 6.8/12.2 MB 7.5 MB/s eta 0:00:01
   --------------------------- ------------ 8.4/12.2 MB 7.6 MB/s eta 0:00:01
   --------------------------------- ------ 10.2/12.2 MB 7.8 MB/s eta 0:00:01
   -------------------------------------- - 11.8/12.2 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------- 12.2/12.2 MB 7.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/632.6 kB ? eta -:--:--
   ---------------------------------------- 632.6/632.6 kB 7.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------------------

In [6]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/124.9 MB 1.9 MB/s eta 0:01:07
   ---------------------------------------- 1.3/124.9 MB 3.7 MB/s eta 0:00:34
   - -------------------------------------- 3.4/124.9 MB 6.1 MB/s eta 0:00:20
   - -------------------------------------- 5.2/124.9 MB 6.4 MB/s eta 0:00:19
   -- ------------------------------------- 6.8/124.9 MB 6.6 MB/s eta 0:00:19
   -- ------------------------------------- 7.6/124.9 MB 6.7 MB/s eta 0:00:18
   -- ------------------------------------- 9.2/124.9 MB 6.1 MB/s eta 0:00:19
   --- ------------------------------------ 10.2/124.9 MB 6.1 MB/s eta 0:00:19
   --- ------------------------------------ 11.5/124.9 MB 6.0 MB/s eta 0:00:19
   ---- ----------------------------------- 12.8/124.9 MB 6.0 MB/s eta 0:00:19
   ---- ----------------------------------- 14.2/124.9 MB 6.0 MB/s eta 0:00

In [7]:
import pandas as pd
import numpy as np
import re
import nltk
import string
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
train_df = pd.read_csv("train_tm/train.csv")
print(train_df.head())


   ID                                              TITLE  \
0   1        Reconstructing Subject-Specific Effect Maps   
1   2                 Rotation Invariance Neural Network   
2   3  Spherical polyharmonics and Poisson kernels fo...   
3   4  A finite element approximation for the stochas...   
4   5  Comparative study of Discrete Wavelet Transfor...   

                                            ABSTRACT  Computer Science  \
0    Predictive models allow subject-specific inf...                 1   
1    Rotation invariance and translation invarian...                 1   
2    We introduce and develop the notion of spher...                 0   
3    The stochastic Landau--Lifshitz--Gilbert (LL...                 0   
4    Fourier-transform infra-red (FTIR) spectra o...                 1   

   Physics  Mathematics  Statistics  Quantitative Biology  \
0        0            0           0                     0   
1        0            0           0                     0   
2        0 

In [18]:
# Text Preprocessing
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sbaty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
train_df['cleaned_text'] = train_df['TITLE'] + " " + train_df['ABSTRACT']
train_df['cleaned_text'] = train_df['cleaned_text'].apply(clean_text)
train_df['cleaned_text'] = train_df['cleaned_text'].apply(remove_stopwords)

In [20]:
# Feature Extraction using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(train_df['cleaned_text'])

In [ ]:
y = train_df[['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Train Random Forest 
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_val)

In [ ]:
# Train XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_val)

C:\Users\sbaty\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [17:57:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [24]:
# Evaluation
print("Random Forest Performance:")
print(classification_report(y_val, rf_preds))

print("XGBoost Performance:")
print(classification_report(y_val, xgb_preds))

Random Forest Performance:
              precision    recall  f1-score   support

           0       0.78      0.83      0.80      1692
           1       0.95      0.71      0.81      1226
           2       0.91      0.62      0.74      1150
           3       0.81      0.47      0.59      1069
           4       0.00      0.00      0.00       122
           5       0.00      0.00      0.00        45

   micro avg       0.85      0.66      0.74      5304
   macro avg       0.57      0.44      0.49      5304
weighted avg       0.83      0.66      0.72      5304
 samples avg       0.74      0.70      0.71      5304

XGBoost Performance:
              precision    recall  f1-score   support

           0       0.81      0.82      0.81      1692
           1       0.91      0.80      0.85      1226
           2       0.86      0.72      0.78      1150
           3       0.79      0.69      0.74      1069
           4       0.61      0.20      0.31       122
           5       0.84      0

C:\Users\sbaty\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sbaty\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\sbaty\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati